# Проведем оценку работы нескольких моделей и выберем лучшую

Проверим следующие модели:
- dummy model (mean by brand_name)
- линейные модели (с регуляризацией)
- случайный лес
- cat boost

Также для передачи в модель данные требуется векторизовать


In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator

from sklearn.linear_model import Ridge, LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool

from ngboost import NGBRegressor
from ngboost.distns import Normal
from scipy.stats import norm

from sklearn.metrics import mean_squared_error


In [2]:
SEED = 42

# Прочитаем данные

In [3]:
df = pd.read_csv("../data/processed/clean_data.csv")

In [4]:
df.head()

,brand_name,priceLog,proc_freq,proc_brand,proc_name,proc_count,videocard,videocard_memory,screen,ssd_volume,ram,hdmi,material,battery_life
0,other,11.156251,1.600000,intel,intel core i5,4.0,интегрированная,0.0,15.0,512,16,True,металл,7.0
1,other,11.561716,2.000000,intel,intel core i5,8.0,geforce rtx,4.0,17.0,512,16,True,пластик,5.0
2,other,11.002100,1.600000,intel,intel core i5,4.0,интегрированная,0.0,15.0,512,8,True,металл,7.0
3,apple,11.849398,3.406613,apple,apple m2,8.0,интегрированная,0.0,15.0,256,8,False,металл,20.0
4,msi,11.775290,2.700000,intel,intel core i5,6.0,geforce rtx,6.0,17.0,512,16,True,металл,5.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand_name        933 non-null    object 
 1   priceLog          933 non-null    float64
 2   proc_freq         933 non-null    float64
 3   proc_brand        933 non-null    object 
 4   proc_name         933 non-null    object 
 5   proc_count        933 non-null    float64
 6   videocard         933 non-null    object 
 7   videocard_memory  933 non-null    float64
 8   screen            933 non-null    float64
 9   ssd_volume        933 non-null    int64  
 10  ram               933 non-null    int64  
 11  hdmi              933 non-null    bool   
 12  material          933 non-null    object 
 13  battery_life      933 non-null    float64
dtypes: bool(1), float64(6), int64(2), object(5)
memory usage: 95.8+ KB


# Разделим на трейн и тест

In [8]:
df_train, df_test = train_test_split(df, shuffle=True, random_state=SEED)

In [9]:
y_train = df_train.priceLog
y_test = df_test.priceLog
X_train = df_train.drop("priceLog", axis=1)
X_test = df_test.drop("priceLog", axis=1)

# Простая модель

In [10]:
class DummyMeanregressor(BaseEstimator):
    """
    Берет среднюю цены по бренду
    """
    def __init__(self):
        self._fitted = False
        
    def fit(self, X, y):
        X_ = X.copy()
        X_["target"] = y
        self._data = X_.groupby("brand_name").target.mean()
        self._fitted = True
    
    def predict(self, X):
        if not self._fitted:
            raise "Model not trained"
        return self._data[X["brand_name"]]
    
    def score(self, X, y):
        return mean_squared_error(self.predict(X), y)
    

In [11]:
def score_model(model, X, y):
    result = list(np.round((-cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error'))**0.5, 5))
    print(f"Cross validation RMSE for {model}")
    print()
    pprint(result)
    print()
    print(f"Mean RMSE: {np.round(np.mean(result), 5)}")
   

In [12]:
score_model(DummyMeanregressor(), X_train, y_train)

Cross validation RMSE for DummyMeanregressor()

[0.56665, 0.64666, 0.57335, 0.60871, 0.60959]

Mean RMSE: 0.60099


# Линейная модель

Для линейной модели сделаем шкалирование признаков и OHE категориальных признаков

### Шкалирование

In [13]:
X_train_linear = X_train.copy()
X_test_linear = X_test.copy()

In [14]:
numerical = X_train_linear.select_dtypes(include=[int, float]).columns.tolist()

In [15]:
scaler = StandardScaler()

In [16]:
X_train_linear[numerical] = scaler.fit_transform(X_train_linear[numerical])

In [17]:
X_test_linear[numerical] = scaler.transform(X_test_linear[numerical])

### Преобразуем категориальные фичи

In [18]:
dv = DictVectorizer(sparse=False)

In [19]:
X_train_linear = dv.fit_transform(X_train_linear.to_dict(orient="records"))
X_test_linear = dv.transform(X_test_linear.to_dict(orient="records"))

### Смотрим результаты

In [20]:
score_model(LinearRegression(), X_train_linear, y_train)

Cross validation RMSE for LinearRegression()

[0.24531, 0.30914, 0.27929, 0.30839, 0.26309]

Mean RMSE: 0.28104


In [21]:
regr_cv = RidgeCV(alphas=[0.1, 1.0, 10.0])

In [22]:
model_cv = regr_cv.fit(X_train_linear, y_train)

In [23]:
print(f"Param alpha = {model_cv.alpha_}")

Param alpha = 1.0


In [24]:
score_model(model_cv, X_train_linear, y_train)

Cross validation RMSE for RidgeCV(alphas=[0.1, 1.0, 10.0])

[0.24495, 0.30818, 0.28006, 0.31012, 0.24817]

Mean RMSE: 0.2783


> У ленейных моделей результаты оказались лучше чем у простой модели, основанных на средних ценах
> Сложно оценить погрешность, в реальных значениях, т.к. мы использовали логарифм цены.
> Но видно существенное улучшение
>
> Также заметим что модель с регуляризацией показала еще более лучший результат (но не значительно)

# Случайный лес

Также требует преобразования категориальных данных, но шкалирование делать уже не нужно

In [53]:
X_train_forest = X_train.copy()
X_test_forest = X_test.copy()

In [54]:
dv = DictVectorizer(sparse=False)
X_train_forest = dv.fit_transform(X_train_forest.to_dict(orient="records"))
X_test_forest = dv.transform(X_test_forest.to_dict(orient="records"))

### Смотрим результаты

In [55]:
for depth in [1,3,7,10, 12, 15]:
    print(f"DEPTH {depth}")
    score_model(RandomForestRegressor(n_estimators=200, max_depth=depth, random_state=SEED), X_train_forest, y_train)


DEPTH 1
Cross validation RMSE for RandomForestRegressor(max_depth=1, n_estimators=200, random_state=42)

[0.42047, 0.52617, 0.4689, 0.51254, 0.46665]

Mean RMSE: 0.47895
DEPTH 3
Cross validation RMSE for RandomForestRegressor(max_depth=3, n_estimators=200, random_state=42)

[0.28884, 0.31856, 0.30888, 0.29269, 0.28551]

Mean RMSE: 0.2989
DEPTH 7
Cross validation RMSE for RandomForestRegressor(max_depth=7, n_estimators=200, random_state=42)

[0.22894, 0.257, 0.24301, 0.19895, 0.21418]

Mean RMSE: 0.22842
DEPTH 10
Cross validation RMSE for RandomForestRegressor(max_depth=10, n_estimators=200, random_state=42)

[0.2261, 0.24968, 0.24514, 0.19129, 0.20566]

Mean RMSE: 0.22357
DEPTH 12
Cross validation RMSE for RandomForestRegressor(max_depth=12, n_estimators=200, random_state=42)

[0.22676, 0.25008, 0.24494, 0.18977, 0.20511]

Mean RMSE: 0.22333
DEPTH 15
Cross validation RMSE for RandomForestRegressor(max_depth=15, n_estimators=200, random_state=42)

[0.22691, 0.24982, 0.2445, 0.18992, 0.2

In [56]:
modelForest = RandomForestRegressor(random_state=SEED, n_estimators=2000)

In [57]:
modelForest.fit(X_train_forest, y_train)

RandomForestRegressor(n_estimators=2000, random_state=42)

In [59]:
mean_squared_error(modelForest.predict(X_test_forest), y_test)**0.5

0.20272778402077868

> Результаты случайного леса оказались еще лучше  
> Оптимальной глубиной для леса оказалась глубина 10 

# CatBoost

Для catboost не будем делать преобразование, но положим все данные в catboost.Pool, указав какие фичи являются категориальными.

In [35]:
cat_feats = list(X_train.select_dtypes('object').columns)
train_dataset = Pool(X_train, y_train, cat_features=cat_feats) 
test_dataset = Pool(X_test, y_test, cat_features=cat_feats) 

C:\Soft\Anaconda\lib\site-packages\catboost\core.py:1135: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight, group_id, group_weight, subgroup_id, pairs_weight, baseline, feature_names, thread_count)


In [43]:
model = CatBoostRegressor(loss_function='RMSE', random_seed=SEED)

In [ ]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}
params, cv_results = model.grid_search(grid, train_dataset, plot=True)

In [49]:
mean_squared_error(model.predict(test_dataset), y_test)**0.5

0.19433217188573745

> Результат catboost еще лучше чем у RandomForest.  
> Но мы его и оптимизировали лучше через gridsearch

# NGBoost

Попробуем модель из вероятностного программирования [NGBoost](https://github.com/stanfordmlgroup/ngboost)  
Она может дать нам не только прогноз, но и доверительные интервалы.

In [27]:
score_model(NGBRegressor(Dist=Normal, random_state=SEED), X_train_forest, y_train)

[iter 0] loss=1.1594 val_loss=0.0000 scale=1.0000 norm=0.8395
[iter 100] loss=0.4862 val_loss=0.0000 scale=2.0000 norm=1.0239
[iter 200] loss=-0.0950 val_loss=0.0000 scale=2.0000 norm=0.8867
[iter 300] loss=-0.3472 val_loss=0.0000 scale=2.0000 norm=0.9048
[iter 400] loss=-0.4496 val_loss=0.0000 scale=2.0000 norm=0.9209
[iter 0] loss=1.1222 val_loss=0.0000 scale=1.0000 norm=0.8207
[iter 100] loss=0.3836 val_loss=0.0000 scale=2.0000 norm=0.9872
[iter 200] loss=-0.1558 val_loss=0.0000 scale=2.0000 norm=0.8877
[iter 300] loss=-0.3747 val_loss=0.0000 scale=2.0000 norm=0.9367
[iter 400] loss=-0.4748 val_loss=0.0000 scale=2.0000 norm=0.9584
[iter 0] loss=1.1347 val_loss=0.0000 scale=1.0000 norm=0.8284
[iter 100] loss=0.4348 val_loss=0.0000 scale=2.0000 norm=1.0125
[iter 200] loss=-0.1407 val_loss=0.0000 scale=2.0000 norm=0.8982
[iter 300] loss=-0.3873 val_loss=0.0000 scale=2.0000 norm=0.9282
[iter 400] loss=-0.4952 val_loss=0.0000 scale=1.0000 norm=0.4752
[iter 0] loss=1.1089 val_loss=0.0000 

In [28]:
modelNG = NGBRegressor(Dist=Normal, random_state=SEED, n_estimators=2000)

In [29]:
modelNG.fit(X_train_forest, y_train)

[iter 0] loss=1.1340 val_loss=0.0000 scale=1.0000 norm=0.8280
[iter 100] loss=0.4223 val_loss=0.0000 scale=2.0000 norm=1.0061
[iter 200] loss=-0.1222 val_loss=0.0000 scale=2.0000 norm=0.8992
[iter 300] loss=-0.3541 val_loss=0.0000 scale=1.0000 norm=0.4668
[iter 400] loss=-0.4572 val_loss=0.0000 scale=1.0000 norm=0.4784
[iter 500] loss=-0.5289 val_loss=0.0000 scale=2.0000 norm=0.9543
[iter 600] loss=-0.5722 val_loss=0.0000 scale=1.0000 norm=0.4691
[iter 700] loss=-0.6053 val_loss=0.0000 scale=1.0000 norm=0.4628
[iter 800] loss=-0.6304 val_loss=0.0000 scale=0.5000 norm=0.2302
[iter 900] loss=-0.6414 val_loss=0.0000 scale=0.2500 norm=0.1143
[iter 1000] loss=-0.6519 val_loss=0.0000 scale=0.0020 norm=0.0009
[iter 1100] loss=-0.6528 val_loss=0.0000 scale=0.0020 norm=0.0009
[iter 1200] loss=-0.6551 val_loss=0.0000 scale=0.5000 norm=0.2286
[iter 1300] loss=-0.6606 val_loss=0.0000 scale=0.0010 norm=0.0004
[iter 1400] loss=-0.6657 val_loss=0.0000 scale=0.0010 norm=0.0004
[iter 1500] loss=-0.6666

NGBRegressor(n_estimators=2000,
             random_state=RandomState(MT19937) at 0x1F06FD32D40)

In [30]:
mean_squared_error(modelNG.predict(X_test_forest), y_test)**0.5

0.20559921257563266

> Модель NG boost рабоает хуже catboost и чуть-чуть хуже случайного леса  
> Но у нее есть возможность предсказывать доверительные интевалы, и давайте воспользуемся этой возможностью

In [60]:
preds = modelNG.pred_param(X_test_forest)

In [61]:
def get_ci(preds, ci_level=0.95):
    lower_bound = (1-ci_level)/2
    z_score = norm.ppf(lower_bound)
    
    mean = preds[:,0]
    lower = preds[:,0] + z_score * np.exp(preds[:,1])
    upper = preds[:,0] - z_score * np.exp(preds[:,1])
    
    return np.c_[mean, lower, upper]
    

In [77]:
preds_with_ci = get_ci(preds)
plot_data = np.round(np.expm1(np.c_[y_test, preds_with_ci]), -2)
result_ngboost = pd.DataFrame(plot_data, columns=["y", "pred", "lower95", "upper95"])

In [78]:
result_ngboost

,y,pred,lower95,upper95
0,160000.0,126200.0,93900.0,169700.0
1,50000.0,40500.0,30500.0,53600.0
2,170000.0,140400.0,104600.0,188600.0
3,30000.0,30000.0,21700.0,41600.0
4,300000.0,258500.0,214800.0,311000.0
...,...,...,...,...
229,185000.0,196800.0,149000.0,259900.0
230,60000.0,64200.0,48000.0,85900.0
231,120000.0,94700.0,68800.0,130300.0
232,75000.0,94100.0,69700.0,127100.0


In [86]:
y_in_CI_rate = np.mean((result_ngboost.y > result_ngboost.lower95) & (result_ngboost.y < result_ngboost.upper95))

In [91]:
print(f"Значение y попадает в 95% доверительные интервал в {y_in_CI_rate*100:.0f}%")

Значение y попадает в 95% доверительные интервал в 89%


> Мы видем, что даже доверительные интервалы предсказаны неплохо, и target в них хорошо попадает  
> Наверное это неплохо